# Results: Halo Orbits, Manifold Tubes, and Re-entry Mapping\n\nThis notebook loads the saved outputs and regenerates the primary figures for the report.

In [1]:
import os\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nfrom cr3bp.constants import EARTH_MOON_MU\nfrom cr3bp.manifolds import sample_manifold\nfrom cr3bp.plotting import plot_orbits_matplotlib, plot_orbits_plotly, export_plotly\nfrom cr3bp.shooting import find_halo_orbit\nfrom cr3bp.poincare import poincare_section\n\nplt.rcParams["figure.figsize"] = (8, 6)\nplt.rcParams["axes.grid"] = True\n\nmu = EARTH_MOON_MU\nos.makedirs("outputs", exist_ok=True)

SyntaxError: unexpected character after line continuation character (1126732484.py, line 1)

In [ ]:
# Load or generate manifolds\ndata_path = os.path.join("outputs", "phase3_manifolds.npz")\nif os.path.exists(data_path):\n    data = np.load(data_path, allow_pickle=True)\n    orbit_states = data["orbit_states"]\n    manifold_trajs = data["manifold_trajectories"]\nelse:\n    halo = find_halo_orbit(mu, x0=0.836914, z0_guess=0.05, vy0_guess=0.25)\n    state0 = halo["state0"]\n    period = float(halo["period"])\n    data = sample_manifold(state0, mu, period, n_samples=60, eps=1e-6, manifold_time=5.0)\n    orbit_states = data["orbit_states"]\n    manifold_trajs = data["manifold_trajectories"]

In [ ]:
# Matplotlib 3D figure\nfig = plot_orbits_matplotlib(orbit_states, manifold_trajs, title="Halo Orbit and Manifold Tubes")\nplt.show()

In [ ]:
# Plotly export\nfig_plotly = plot_orbits_plotly(orbit_states, manifold_trajs, title="Halo Orbit and Manifold Tubes")\nexport_plotly(fig_plotly, os.path.join("outputs", "results_manifolds.html"))

In [ ]:
# Poincare section (re-entry mapping)\nfrom cr3bp.constants import EARTH_MOON_DISTANCE_KM, EARTH_RADIUS_KM\nradius_nd = (EARTH_RADIUS_KM + 120.0) / EARTH_MOON_DISTANCE_KM\nsection = poincare_section(manifold_trajs, mu, radius_nd)\nprint("Hits:", len(section["hits"]))\nprint("Feasible:", len(section["feasible"]))

In [ ]:
# Simple lat/lon scatter (inertial)\nif len(section["feasible"]) > 0:\n    lat = np.degrees(section["feasible"][:, 8])\n    lon = np.degrees(section["feasible"][:, 9])\n    plt.figure()\n    plt.scatter(lon, lat, s=12)\n    plt.xlabel("Longitude (deg)")\n    plt.ylabel("Latitude (deg)")\n    plt.title("Feasible Re-entry Points (Inertial)")\n    plt.show()

In [ ]:
# Plotly globe view if Phase 4 outputs exist
import plotly.graph_objects as go
p4_path = os.path.join("outputs", "phase4_poincare.npz")
if os.path.exists(p4_path):
    p4 = np.load(p4_path)
    feasible = p4["feasible"]
    if feasible.size > 0:
        lat = np.degrees(feasible[:, 8])
        lon = np.degrees(feasible[:, 9])
        lat_ecef = feasible[:, 10]
        lon_ecef = feasible[:, 11]
        fig = go.Figure()
        fig.add_trace(go.Scattergeo(lat=lat, lon=lon, mode="markers", name="Inertial"))
        if np.isfinite(lat_ecef).all():
            fig.add_trace(go.Scattergeo(lat=np.degrees(lat_ecef), lon=np.degrees(lon_ecef), mode="markers", name="Earth-fixed"))
        fig.update_layout(geo=dict(projection_type="orthographic"))
        fig.show()
